# Monthly stats from warehouse


We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
import cloud_sql_connector
import pandas as pd
from datetime import datetime, timedelta
from cloud_sql_connector import DBConfig, getconn
import pg8000
import sqlalchemy

ModuleNotFoundError: No module named 'cloud_sql_connector'

This will create the connection to the database

In [ ]:
# Initialize DBConfig for Cloud SQL connection
config = DBConfig(
    instance_name=os.getenv('DB_INSTANCE_CONNECTION_NAME'),
    database=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    ip_type="public",
    schema="auth"
)

def get_conn():
    """Create a connection to Google Cloud SQL using the custom cloud-sql-connector."""
    return getconn(config)

# Create SQLAlchemy engine using Cloud SQL connector
engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_conn,
)

print("Cloud SQL engine created successfully!")

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT 1"))
        print("✅ DB connection OK:", result.fetchone())
except Exception as e:
    print("❌ Connection failed:", e)

In [ ]:
#%% File setup
datestr = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
filename =  os.path.join(os.getcwd(), r'data/') + 'dwh_monthly_stats_' + datestr + '.csv'


In [ ]:
#%% Accounts created in the current month
query = """
SELECT 
    TO_CHAR(created AT TIME ZONE 'America/Vancouver', 'Month YYYY') AS month,
    COUNT(*) AS record_count
FROM auth.orgs
WHERE DATE_TRUNC('month', created AT TIME ZONE 'America/Vancouver')
      = DATE_TRUNC('month', NOW() AT TIME ZONE 'America/Vancouver')
GROUP BY month
"""
stat_accounts_monthly_completed = pd.read_sql(query, engine)

with open(filename, 'w') as f:
    stat_accounts_monthly_completed.insert(0, "Monthly Number of New Accounts:", [''], True)
    stat_accounts_monthly_completed.to_csv(f, sep=',', encoding='utf-8', index=False)


In [ ]:
#%% Names request submitted in the current month
query = """
SELECT 
    TO_CHAR(submitted_date AT TIME ZONE 'America/Vancouver', 'FMMonth YYYY') AS month,
    COUNT(*) AS record_count
FROM namex.requests
WHERE DATE_TRUNC('month', submitted_date AT TIME ZONE 'America/Vancouver')
      = DATE_TRUNC('month', NOW() AT TIME ZONE 'America/Vancouver')
GROUP BY month
"""
stat_nr_total_completed = pd.read_sql(query, engine)

with open(filename, 'a') as f:      
    f.write('\n\n')
    stat_nr_total_completed.insert(0, "Monthly Number of New NRs:", [''], True)
    stat_nr_total_completed.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
#%% Monthly SP/GP registrations
query = """
SELECT 
    TO_CHAR(founding_date AT TIME ZONE 'America/Vancouver', 'FMMonth YYYY') AS month,
    COUNT(*) AS record_count
FROM business.businesses
WHERE DATE_TRUNC('month', founding_date AT TIME ZONE 'America/Vancouver')
      = DATE_TRUNC('month', NOW() AT TIME ZONE 'America/Vancouver')
  AND legal_type IN ('SP', 'GP')
GROUP BY month
"""
stat_sp_gp_registrations_completed = pd.read_sql(query, engine)

with open(filename, 'a') as f:      
    f.write('\n\n')
    stat_sp_gp_registrations_completed.insert(0, "Monthly Number of New NRs:", [''], True)
    stat_sp_gp_registrations_completed.to_csv(f, sep=',', encoding='utf-8', index=False)